In [95]:
import geojson
import pandas
import json
from shapely.geometry import shape, Point
from collections import OrderedDict
import csv
import numpy as np 

In [11]:
path = 'TrentoTreeMap/data/geo_data_trees.geojson'

In [14]:
with open(path, 'r') as data:
    loaded_json = geojson.load(data)
    features = loaded_json['features']
    tree_names = set()
    for feature in features:
        properties = feature['properties']
        tree_names.add(properties['Name'])
    
    result = {}
    for tree in tree_names:
        result[tree] = {'count': 0, 'total_canopy_cover': 0}
        
    for feature in features: 
        properties = feature['properties']
        result[properties['Name']]['count'] += 1
        result[properties['Name']]['total_canopy_cover'] += float(properties['Canopy Cover (m2)'])
    
    sorted_dictionary = OrderedDict(sorted(result.items(), key=lambda item: item[1]['count'], reverse=True))
    
    result = []
    for key, value in sorted_dictionary.items():
        elem = sorted_dictionary[key]
        elem['mean_canopy_cover'] = elem['total_canopy_cover'] / elem['count']
        #elem.pop('total_canopy_cover')
        result += [[key, elem['count'], elem['mean_canopy_cover']]]
        
    with open('data1.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['name', 'count', 'mean_canopy_cover'])
        for line in result:
            writer.writerow(line)

In [16]:
circoscrizioni_path = 'TrentoTreeMap/data/circoscrizioni.json'

In [113]:
with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        circoscrizione = feature['properties']['nome']
                        try:
                            curr_count = result[circoscrizione][tree_name]
                            result[circoscrizione][tree_name] += 1
                        except KeyError:
                            result[circoscrizione][tree_name] = 0
            except TypeError:
                continue
            
        with open('data2.csv', 'w+') as dump:
            tree_names = set()
            for feature in loaded_json['features']:
                properties = feature['properties']
                tree_names.add(properties['Name'])
                
            writer = csv.writer(dump)
            headers = ['circoscrizione'] + list(tree_names)
            writer.writerow(headers)
            
            for k, v in result.items():
                to_write = [k]
                for nome_albero in headers[1:]:
                    try:
                        to_write += [result[k][nome_albero]]
                    except KeyError:
                        to_write += [0]
                
                writer.writerow(to_write)

In [117]:
df = pandas.read_csv('data2.csv')
df.loc[:, df.iloc[0] != 0].iloc[0]

circoscrizione          SARDAGNA
Robinia pseudoacacia           5
Acer pseudoplatanus           10
Parrotia persica               8
Acer platanoides               1
Cupressus                      5
Name: 0, dtype: object